In [ ]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.datasets import CocoDetection
from torchvision.transforms import functional as F
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!cp -r "/content/drive/MyDrive/tree_coco.zip" /content/


In [ ]:
!unzip /content/tree_coco.zip -d /content/


Archive:  /content/tree_coco.zip
   creating: /content/tree_coco/
   creating: /content/tree_coco/test/
  inflating: /content/tree_coco/test/17197-824e56_jpg.rf.b3bbd7effc3f1a8a722395cc6e0752c7.jpg  
  inflating: /content/tree_coco/test/18375-fed59e_jpg.rf.d666e2bfada88ded2a830655dde69769.jpg  
  inflating: /content/tree_coco/test/20041-da5bae_jpg.rf.8d611f377d259c4a91e12fd4c46c18c1.jpg  
  inflating: /content/tree_coco/test/26273-9b3328_jpg.rf.dca8f4e8e621ae2dec4bdb4ef24ba30d.jpg  
  inflating: /content/tree_coco/test/29558-d89820_jpg.rf.fd18923b022545ae4e484872c60408b8.jpg  
  inflating: /content/tree_coco/test/41260-fc875c_jpg.rf.29c2f05f9106817aba7d8870669b589c.jpg  
  inflating: /content/tree_coco/test/46088-37f06b_jpg.rf.c9bbe762f144cf247fee910fdacb934b.jpg  
  inflating: /content/tree_coco/test/46213-f7249a_jpg.rf.67d2a1a336b8fcb9387556f7bfd41998.jpg  
  inflating: /content/tree_coco/test/46739-5366b7da79ee_jpg.rf.a9af69a4bea2cad68786cca3069ddecf.jpg  
  inflating: /content/tree

In [ ]:
class CocoTransform:
    def __call__(self, image, target):
        image = F.to_tensor(image)
        return image, target

In [ ]:
def get_coco_dataset(img_dir, ann_file):
    return CocoDetection(
        root=img_dir,
        annFile=ann_file,
        transforms=CocoTransform()
    )

train_dataset = get_coco_dataset(
    img_dir="/content/tree_coco/train",
    ann_file="/content/tree_coco/train/_annotations.coco.json"
)


val_dataset = get_coco_dataset(
    img_dir="/content/tree_coco/valid",
    ann_file="/content/tree_coco/valid/_annotations.coco.json"
)




train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [ ]:

def get_model(num_classes):

    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="COCO_V1")


    in_features = model.roi_heads.box_predictor.cls_score.in_features


    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

In [ ]:
num_classes = 2
model = get_model(num_classes)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:01<00:00, 86.8MB/s]


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

In [ ]:
def train_one_epoch(model, optimizer, data_loader, device, epoch):
    model.train()
    for images, targets in data_loader:
        # Move images to the device
        images = [img.to(device) for img in images]

        # Validate and process targets
        processed_targets = []
        valid_images = []
        for i, target in enumerate(targets):
            boxes = []
            labels = []
            for obj in target:
                # Extract bbox
                bbox = obj["bbox"]  # Format: [x, y, width, height]
                x, y, w, h = bbox

                # Ensure the width and height are positive
                if w > 0 and h > 0:
                    boxes.append([x, y, x + w, y + h])  # Convert to [x_min, y_min, x_max, y_max]
                    labels.append(obj["category_id"])

            # Only process if there are valid boxes
            if boxes:
                processed_target = {
                    "boxes": torch.tensor(boxes, dtype=torch.float32).to(device),
                    "labels": torch.tensor(labels, dtype=torch.int64).to(device),
                }
                processed_targets.append(processed_target)
                valid_images.append(images[i])  # Add only valid images

        # Skip iteration if no valid targets
        if not processed_targets:
            continue

        # Ensure images and targets are aligned
        images = valid_images

        # Forward pass
        loss_dict = model(images, processed_targets)
        losses = sum(loss for loss in loss_dict.values())

        # Backpropagation
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

    print(f"Epoch [{epoch}] Loss: {losses.item():.4f}")

In [ ]:
num_epochs = 5
for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, train_loader, device, epoch)
    lr_scheduler.step()

    # Save the model's state dictionary after every epoch
    model_path = f"fasterrcnn_resnet50_epoch_{epoch + 1}.pth"
    torch.save(model.state_dict(), model_path)
    print(f"Model saved: {model_path}")

KeyboardInterrupt: 

In [ ]:

def get_model(num_classes):

    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model



num_classes = 2

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')



model = get_model(num_classes)
model.load_state_dict(torch.load("/content/fasterrcnn_resnet50_epoch_5.pth"))
model.to(device)
model.eval()

def prepare_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image_tensor = F.to_tensor(image).unsqueeze(0)
    return image_tensor.to(device)




image_path = "/content/test3.jpg"
image_tensor = prepare_image(image_path)

with torch.no_grad():
    prediction = model(image_tensor)


COCO_CLASSES = {0: "Background", 1: "log"}

def get_class_name(class_id):
    return COCO_CLASSES.get(class_id, "Unknown")


def draw_boxes(image, prediction, fig_size=(10, 10)):
    boxes = prediction[0]['boxes'].cpu().numpy()
    labels = prediction[0]['labels'].cpu().numpy()
    scores = prediction[0]['scores'].cpu().numpy()


    threshold = 0.5
    num_boxes = sum(score > threshold for score in scores)




    plt.figure(figsize=fig_size)

    for box, label, score in zip(boxes, labels, scores):
        if score > threshold:
            x_min, y_min, x_max, y_max = box
            class_name = get_class_name(label)
            plt.imshow(image)
            plt.gca().add_patch(plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min,
                                              linewidth=2, edgecolor='r', facecolor='none'))
            plt.text(x_min, y_min, f"{class_name} ({score:.2f})", color='r')


    plt.axis('off')
    plt.show()
    print(f"Number of Wood logs detected: {num_boxes}")
draw_boxes(Image.open(image_path), prediction, fig_size=(10,10))


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FileNotFoundError: [Errno 2] No such file or directory: '/content/fasterrcnn_resnet50_epoch_5.pth'